In [ ]:
!pip install torch opencv-python pandas
!pip install kaggle

In [ ]:
!kaggle datasets download -d ayushspai/sportsmot -q
!unzip -q sportsmot.zip -d SportsMOT
!rm sportsmot.zip
!cp -r /content/SportsMOT/sportsmot_publish/dataset/val/* /content/SportsMOT/sportsmot_publish/dataset/train > /dev/null 2>&1

In [ ]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, ColorJitter

import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Dataset

In [ ]:
import os

def load_football_sequences(splits_dir):
    football_file = os.path.join(splits_dir, "football.txt")
    with open(football_file, "r") as f:
        all_sequences = f.read().splitlines()

    return all_sequences

In [ ]:
class FootballDataset(Dataset):
    def __init__(self, root_dir, split="train", transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform

        splits_dir = os.path.join(root_dir, "splits_txt")
        self.football_sequences = load_football_sequences(splits_dir)

        self.sequences = []
        for seq in self.football_sequences:
            seq_path = os.path.join(root_dir, "dataset", split, seq)
            if os.path.exists(seq_path):
                self.sequences.append(seq)

        self.annotations = {}
        for seq in self.sequences:
            gt_file = os.path.join(root_dir, "dataset", split, seq, "gt", "gt.txt")
            if os.path.exists(gt_file):
                self.annotations[seq] = self._load_annotations(gt_file)

        self.default_transform = Compose([
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def _load_annotations(self, gt_file):
        df = pd.read_csv(gt_file, header=None)
        df.columns = [
            "frame_id", "object_id", "x", "y", "w", "h",
            "conf", "class_id", "visibility"
        ]

        annotations = {}
        for frame_id, group in df.groupby("frame_id"):
            annotations[frame_id] = {
                "boxes": group[["x", "y", "w", "h"]].values.astype("float32"),
                "object_ids": group["object_id"].values.astype("int32"),
                "classes": group["class_id"].values.astype("int32")
            }
        return annotations

    def __len__(self):
        return sum(len(ann) for ann in self.annotations.values())

    def __getitem__(self, idx):
        for seq in self.sequences:
            seq_ann = self.annotations[seq]
            if idx < len(seq_ann):
                frame_id = list(seq_ann.keys())[idx]
                break
            idx -= len(seq_ann)
        else:
            raise IndexError("Index out of range")

        img_path = os.path.join(self.root_dir, "dataset", self.split, seq, "img1", f"{frame_id:06d}.jpg")
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        ann = self.annotations[seq][frame_id]

        if self.transform:
            image = self.transform(image)
        else:
            image = self.default_transform(image)

        return image, {
            "boxes": torch.tensor(ann["boxes"]),
            "object_ids": torch.tensor(ann["object_ids"]),
            "classes": torch.tensor(ann["classes"]),
            "frame_id": frame_id,
            "sequence": seq
        }

## Data Loader

In [ ]:
def custom_collate(batch):
    images = torch.stack([item[0] for item in batch])
    targets = [item[1] for item in batch]
    return images, targets

In [ ]:
dataset_dir = "/content/SportsMOT/sportsmot_publish"

transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = FootballDataset(root_dir=dataset_dir, split="train", transform=transform)

train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(
    dataset, [train_size, valid_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate, num_workers=2)

In [ ]:
len(train_loader), len(test_loader), len(valid_loader)

## Object Detection